In [51]:
# 로지스틱 회귀
# k-최근접 이웃 알고리즘 한계 알아보기

# 데이터 불러오기
import pandas as pd
df = pd.read_csv('data/Fish.csv')
df['Species'].unique() # 유니크한 값만 출력하기

# 일부 컬럼만 추출
fish_input = df[['Weight', 'Length2', 'Length3', 'Height', 'Width']]
fish_input.head() # 길이, 무게 등 데이터 봤을 때 이게 도미인지, 숭어인지 등 맞추는 것 할 것

fish_target = df[['Species']] # 인덱스에 따라 뭐가 숭어/도미인지 정답만 뺀 것 -> target은 정답(이름) 데이터만 가짐

# 훈련/테스트 셋 만들기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target)

# 표준화하기: 무게, 길이 간 단위 등이 너무 차이나므로 -> 표준화해야 함
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input, train_target)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)
    # input만 표준점수로 바꾸는 것임 -> target은 str이니까

# 학습시키기
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier()
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))
# > 0.8571428571428571
# 0.75
# k-최근접 사용했을 때, 정합률 좋지 못함

kn.predict_proba(test_scaled)
    # k-최근접 이웃 알고리즘: 주변에 뭐가 많은지 다수결 원칙으로 가중치 둬서 정답 도출함
        # 단순히 주변에 뭐가 있는지가 중요한 것 -> 무게, 길이에 따라 두께 몇 일 확률 계산하는 게 아닌, 주변의 데이터 무엇인지 다수결인 것

0.8319327731092437
0.775


C:\Users\1-08\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


array([[0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0.8, 0. , 0.2, 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.8, 0. , 0. , 0. , 0.2],
       [0. , 0.4, 0.2, 0. , 0.4, 0. , 0. ],
       [0. , 0.4, 0. , 0. , 0.6, 0. , 0. ],
       [0. , 0. , 0.6, 0. , 0.2, 0. , 0.2],
       [0. , 0. , 0.6, 0. , 0.4, 0. , 0. ],
       [0. , 0. , 0.2, 0. , 0.8, 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.8, 0. , 0. , 0. , 0.2],
       [0. , 0. , 0.8, 0. , 0. , 0. , 0.2],
       [0. , 0. , 0.4, 0. , 0.6, 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.8, 0. , 0.2, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.4, 0.6, 0. , 0. , 0. ,

In [49]:
train_target

,Species
46,Roach
56,Whitefish
68,Parkki
134,Pike
155,Smelt
...,...
126,Perch
73,Perch
24,Bream
95,Perch


In [53]:
# 로지스틱 회귀 분석

# 1. 이진 분류: 도미/빙어만 추출
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bs = train_scaled[bream_smelt_indexes['Species']] # 현재 train_scaled는 [1, -1, - ...] 이런 식으로 돼 있는데, 이중 true(1)인 것만 고르기 위함
target_bs = train_target[bream_smelt_indexes['Species']]
    # train_scaled, train_target 모두 도미/빙어만 이진 분류로 필터링

# 2. 로지스틱 회귀 모델 불러오기
from sklearn.linear_model import LogisticRegression
import numpy as np

lr = LogisticRegression()
lr.fit(train_bs, target_bs) # 이진분류한 것을 토대로 학습
lr.predict(train_bs[:5]) # 이진분류된 학습 데이터를 앞에서 5번째까지만 예측하기
lr.predict_proba(train_bs[:5]).round(3) # 확률이 어떻길래 각 값을 도미/빙어로 판단한 것인지 확률 보기
    # round: numpy의 반올림하는 함수
# 로지스틱 회귀 원리: 길이, 높이, 무게 등 데이터를 기반으로 연산해서 이를 확률로 변환 -> 그래서 도미/빙어 판단    

C:\Users\1-08\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[0.985, 0.015],
       [1.   , 0.   ],
       [0.979, 0.021],
       [0.999, 0.001],
       [0.968, 0.032]])

In [52]:
bream_smelt_indexes

,Species
65,False
90,False
127,False
153,True
108,False
...,...
146,True
50,False
154,True
31,True


In [33]:
# 다중분류: 7마리 모두 학습
lr = LogisticRegression(C=20, max_iter=1000) # 성능 높이기 위해 옵션 지정
    # max: 반복 횟수 정하는 것(기본값 100) -> 높을수록 성능 좋아짐
lr.fit(train_scaled, train_target) # 7마리 모두 있는 train_scaled 넣은 것
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))
lr.predict_proba(test_scaled[:5]).round(3)

0.907563025210084
0.975


C:\Users\1-08\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[0.211, 0.   , 0.359, 0.164, 0.007, 0.   , 0.259],
       [0.984, 0.005, 0.   , 0.   , 0.   , 0.   , 0.011],
       [0.   , 0.   , 0.004, 0.996, 0.   , 0.   , 0.   ],
       [0.997, 0.001, 0.   , 0.   , 0.   , 0.   , 0.002],
       [0.01 , 0.924, 0.001, 0.   , 0.038, 0.   , 0.027]])

In [37]:
lr.coef_.shape

(7, 5)